In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb
#pdb.set_trace()
from ipywidgets import IntProgress
from IPython.display import display
import os
from selenium.webdriver.support import expected_conditions
from IPython.display import clear_output

In [3]:
TIEMPO_DE_ESPERA = 5.5

start_ = datetime.datetime.now()
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')

user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument("--window-size=1825x1844")
# options.headless = True
browser = webdriver.Chrome( options = chrome_options)


In [4]:
URLS = [['Mujer','https://www2.hm.com/content/hmonline/en_us/women/products/view-all.html/'],
       ['Hombre','https://www2.hm.com/en_us/men/products/view-all.html'],
       ['Kids','https://www2.hm.com/en_us/kids/products/view-all.html']]

In [36]:
paginacion_total = []
for URL in URLS:
    browser.get(URL[1])
    
    if URL[0] =='Kids':
        paginacion_total += browser.execute_script('''

            var cantidad = document.querySelector("#menu-links > li:nth-child(3)").getElementsByClassName('item').length;

            var arn =[]
            var arl =[]

            for (i=0;i<cantidad;i++){


                arn.push(document.querySelector("#menu-links > li:nth-child(3)").getElementsByClassName('item')[i].textContent.trim());

                arl.push('https://www2.hm.com'.concat(document.querySelector("#menu-links > li:nth-child(3)").getElementsByClassName('item')[i].getElementsByTagName('a')[0].getAttribute('href')));
            }
             function zip(a, b) {
                              var arr = [];
                              for (var key in a) arr.push([a[key], b[key]]);
                              return arr;
                            }
            return zip(arl,arn);

            ''')
    else:
            
        paginacion_total += browser.execute_script('''

        var cantidad = document.querySelector("#menu-links > li:nth-child(4)").getElementsByClassName('item').length;

        var arn =[]
        var arl =[]

        for (i=0;i<cantidad;i++){


            arn.push(document.querySelector("#menu-links > li:nth-child(4)").getElementsByClassName('item')[i].textContent.trim());

            arl.push('https://www2.hm.com'.concat(document.querySelector("#menu-links > li:nth-child(4)").getElementsByClassName('item')[i].getElementsByTagName('a')[0].getAttribute('href')));
        }
         function zip(a, b) {
                          var arr = [];
                          for (var key in a) arr.push([a[key], b[key]]);
                          return arr;
                        }
        return zip(arl,arn);

        ''')

for paginacion in paginacion_total:
    
    if paginacion[0].find('/men') != -1 : paginacion.append('Men')
    if paginacion[0].find('/kids') != -1 : paginacion.append('Kids')
    if paginacion[0].find('/women') != -1 : paginacion.append('Women')
        

In [40]:
for i in paginacion_total:
    print(i[2])
    break

Women


In [64]:
URL

['https://www2.hm.com/en_us/kids/products/newborn-size-0-9m.html',
 'Newborn 0-9m',
 'Kids']

In [98]:
copyL = copyL[1:].copy()

In [83]:
URL

['https://www2.hm.com/en_us/kids/products/baby-boy-size-4m-4y.html',
 'Baby Boys 4m-4Y',
 'Kids']

In [96]:
URL

['https://www2.hm.com/en_us/kids/products/baby-exclusive.html',
 'Baby Exclusive 0m-4Y',
 'Kids']

In [97]:
copyL[1:]

[['https://www2.hm.com/en_us/kids/products/baby-exclusive.html',
  'Baby Exclusive 0m-4Y',
  'Kids'],
 ['https://www2.hm.com/en_us/kids/products/girls-size-18m-10y.html',
  'Girls 1 1/2-10Y',
  'Kids'],
 ['https://www2.hm.com/en_us/kids/products/boys-size-18m-10y.html',
  'Boys 1 1/2-10Y',
  'Kids'],
 ['https://www2.hm.com/en_us/kids/products/girls-size-8-14y.html',
  'Girls 8 - 14+ years',
  'Kids'],
 ['https://www2.hm.com/en_us/kids/products/boys-size-8-14y.html',
  'Boys 8 - 14+ years',
  'Kids']]

In [100]:
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument("--window-size=1825x1844")
# options.headless = True
browser = webdriver.Chrome( options = chrome_options)

In [ ]:
#nocargaron = []
#lista_productos = []


for URL in copyL:
    
    #requests de la url //ropa mujer//hombre//kids
    browser.get(URL[0])
    

    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:

        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        try:
            #boton para cargar más items
            browser.find_element_by_class_name('load-more-products').find_element_by_tag_name('button').click()
        except:
            pass
        time.sleep(TIEMPO_DE_ESPERA)
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(1)

    CANTIDAD_PRODUCTOS = browser.execute_script('''
    var x = document.getElementsByClassName('product-item').length;
    return x;''')
    style = {'description_width': 'initial'}
    test = IntProgress(style=style, min = 0, max = CANTIDAD_PRODUCTOS, description=URL[1]+','+URL[2], bar_style='danger',)    
    display(test)
    #CANTIDAD DE PRODUCTOS
    #_ no lo uso porque deja de estar en el DOM ya que hago click y se pierde el elemento, solo necesito el indice,
    for index_producto in range(CANTIDAD_PRODUCTOS):
        
        
        
        try:
            time.sleep(3)
            browser.execute_script(f'''
            document.getElementsByClassName('hm-product-item')[{index_producto}].getElementsByTagName('a')[0].click();
            ''')
        except:
            continue

        try:
            #WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.CLASS_NAME,'group')))
            WebDriverWait(browser, 90).until(EC.presence_of_element_located((By.CLASS_NAME,'mini-slider')))
        except:
            browser.get(browser.current_url)
            time.sleep(5)

        try:

            cantidad_colores = browser.execute_script('''
                            var cant = document.getElementsByClassName('group')[0].getElementsByClassName('list-item').length;
                            return cant;
                            ''')
        except:
            nocargaron.append([browser.current_url])
            browser.execute_script('history.back();')
            continue




        for color in range(cantidad_colores):


            #click en el color.
            browser.execute_script(f'''
                    document.getElementsByClassName('group')[0].getElementsByClassName('list-item')[{color}].getElementsByTagName('a')[0].click();
                    ''')


            nombre_color = browser.execute_script(f'''
                            var nombre = document.getElementsByClassName('group')[0].getElementsByClassName('list-item')[{color}].getElementsByTagName('a')[0].getAttribute('title');
                            return nombre;
                            ''')

            id_producto = browser.execute_script(f'''
                            var id = document.getElementsByClassName('group')[0].getElementsByClassName('list-item')[{color}].getElementsByTagName('a')[0].getAttribute('data-articlecode');
                            return id;
                            ''')

            descripcion = browser.execute_script('''
                            var desc = document.getElementsByClassName('primary product-item-headline')[0].textContent.trim();
                            return desc;
                            ''')

            precio = browser.execute_script('''
                        var precio = document.getElementsByClassName('price parbase')[0].textContent.trim();
                        return precio;
                        ''')

            url_producto = browser.current_url

            img = browser.execute_script('''
                    var img = document.getElementsByClassName('product-detail-main-image-container')[0].getElementsByTagName('img')[0].getAttribute('src');
                    return img;
                    ''')

            lista_productos.append([id_producto,
                                    descripcion,
                                    nombre_color,
                                    URL[1],
                                    precio,
                                    URL[2],
                                    url_producto,
                                    img])

        browser.execute_script('history.back();')
        test.value +=1
        test.description = f"± {int(CANTIDAD_PRODUCTOS)*15}: {test.value} / {CANTIDAD_PRODUCTOS}"

    test.bar_style='success'
    clear_output() 

end_ = datetime.datetime.now()


IntProgress(value=0, bar_style='danger', description='Girls 1 1/2-10Y,Kids', max=2058, style=ProgressStyle(des…

In [ ]:
1152/1328
10:49..

In [55]:
copyL = paginacion_total[2:].copy()

In [51]:
URL[0]

'https://www2.hm.com/en_us/women/products/shirts-blouses.html'

In [48]:
paginacion_total

[['https://www2.hm.com/en_us/women/products/dresses.html', 'Dresses', 'Women'],
 ['https://www2.hm.com/en_us/women/products/tops.html', 'Tops', 'Women'],
 ['https://www2.hm.com/en_us/women/products/shirts-blouses.html',
  'Shirts & Blouses',
  'Women'],
 ['https://www2.hm.com/en_us/women/products/cardigans-sweaters.html',
  'Cardigans & Sweaters',
  'Women'],
 ['https://www2.hm.com/en_us/women/products/jackets-coats.html',
  'Jackets & Coats',
  'Women'],
 ['https://www2.hm.com/en_us/women/products/blazers-vests.html',
  'Blazers',
  'Women'],
 ['https://www2.hm.com/en_us/women/products/pants.html', 'Pants', 'Women'],
 ['https://www2.hm.com/en_us/women/products/jeans.html', 'Jeans', 'Women'],
 ['https://www2.hm.com/en_us/women/products/hoodies-sweatshirts.html',
  'Hoodies & Sweatshirts',
  'Women'],
 ['https://www2.hm.com/en_us/women/products/shoes.html', 'Shoes', 'Women'],
 ['https://www2.hm.com/en_us/women/products/basics.html', 'Basics', 'Women'],
 ['https://www2.hm.com/en_us/women

In [57]:
copyL

[['https://www2.hm.com/en_us/women/products/shirts-blouses.html',
  'Shirts & Blouses',
  'Women'],
 ['https://www2.hm.com/en_us/women/products/cardigans-sweaters.html',
  'Cardigans & Sweaters',
  'Women'],
 ['https://www2.hm.com/en_us/women/products/jackets-coats.html',
  'Jackets & Coats',
  'Women'],
 ['https://www2.hm.com/en_us/women/products/blazers-vests.html',
  'Blazers',
  'Women'],
 ['https://www2.hm.com/en_us/women/products/pants.html', 'Pants', 'Women'],
 ['https://www2.hm.com/en_us/women/products/jeans.html', 'Jeans', 'Women'],
 ['https://www2.hm.com/en_us/women/products/hoodies-sweatshirts.html',
  'Hoodies & Sweatshirts',
  'Women'],
 ['https://www2.hm.com/en_us/women/products/shoes.html', 'Shoes', 'Women'],
 ['https://www2.hm.com/en_us/women/products/basics.html', 'Basics', 'Women'],
 ['https://www2.hm.com/en_us/women/products/skirts.html', 'Skirts', 'Women'],
 ['https://www2.hm.com/en_us/women/products/swimwear.html',
  'Swimwear',
  'Women'],
 ['https://www2.hm.com/e

In [24]:
browser.save_screenshot('asd.png')

True

In [31]:
browser.quit()

In [25]:
print(browser.current_url)

https://www2.hm.com/en_us/productpage.0830770003.html


In [52]:
lista_productos[-2]

['0771826003',
 'Rib-knit Wrap Dress',
 'Red',
 'Dresses',
 '$59.99',
 'Mujer',
 'https://www2.hm.com/en_us/productpage.0771826003.html',
 '//lp2.hm.com/hmgoepprod?set=source[/4d/fe/4dfe78723f0547659d158a99c65a3fa859c88425.jpg],origin[dam],category[ladies_dresses_mididresses],type[LOOKBOOK],res[m],hmver[1]&call=url[file:/product/main]']

In [ ]:
browser.execute_script(f'''

$("#dropdown-colorWithNames label span:contains('{CLICK_COLOR}')").click()

''')

In [16]:
len(lista_productos)

23598

In [17]:
browser.quit()

In [18]:
df = pd.DataFrame(lista_productos)

In [19]:
df.rename(columns={0:'sku',
                  1:'descripcion',
                  2:'color',
                  3:'tipo',
                  4:'precio',
                  5:'sexo',
                  6:'url',
                  7:'jpg'},inplace=True)

In [20]:
df['precio'] = df['precio'].apply(lambda x: float(x.split()[1]))

In [21]:
df.to_excel(f'./Salida/HyM{str(fecha)}.xlsx')